In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from astropy import units as u 

from snewpy.neutrino import Flavor, MassHierarchy, MixingParameters
from snewpy.models.ccsn import Nakazato_2013
from snewpy.flavor_transformation import AdiabaticMSW, NonAdiabaticMSWH, \
                                         TwoFlavorDecoherence, ThreeFlavorDecoherence, \
                                         NeutrinoDecay, AdiabaticMSWes, NonAdiabaticMSWes

mpl.rc('font', size=18)
%matplotlib inline

In [ ]:
model = Nakazato_2013(progenitor_mass=20*u.solMass, revival_time=100*u.ms, metallicity=0.004, eos='shen')
model

In [ ]:
def plot_total_flux(model, xform_nmo, xform_imo):
    """Plot initial and oscillated neutrino luminosities.
    
    Parameters
    ----------
    model : SupernovaModel
        An input model from a CCSN simulation.
    flav_xform : FlavorTransformation
        A FlavorTransformation subclass; used to create an instance.
    """
    
    energies = np.linspace(0.1,10.1,110) * u.MeV   
    D=10* u.kpc
    d = (10*u.kpc).to('cm').value # distance to SN
    mass=0.2* u.eV    
    times = model.get_time()
    burst_epoch = times <= 0.1*u.s
    accretion_epoch = (times > 0.1*u.s) & (times <= 0.5*u.s)
    cooling_epoch = (times > 0.5*u.s) & (times <= 10*u.s)
    
    ilum = {}
    olum_nmo = {}
    olum_imo = {}
    olum_nmo_tof={}
    olum_imo_tof={}
    
    for flavor in Flavor:
        ilum[flavor] = np.zeros(len(times))
        olum_nmo[flavor] = np.zeros(len(times))
        olum_imo[flavor] = np.zeros(len(times))
        olum_nmo_tof[flavor] = np.zeros(len(times))
        olum_imo_tof[flavor] = np.zeros(len(times))

    # Compute the transformed and untransformed flux at each time.
    for i, t in enumerate(times):
        ispec = model.get_initial_spectra(t, energies)
        ospec_nmo = model.get_transformed_spectra(t, energies, xform_nmo)
        ospec_imo = model.get_transformed_spectra(t, energies, xform_imo)
        ospec_nmo_tof = model.get_delayed_flux(D, t, energies, mass, xform_nmo)
        #ospec_imo_tof = model.get_delayed_flux(D, t, energies, mass, xform_imo)
        

        for flavor in Flavor:
            for j, E in enumerate(energies):
                ispec[flavor][j] /= (4.*np.pi*d**2)
                ospec_nmo[flavor][j] /= (4.*np.pi*d**2)
                ospec_imo[flavor][j] /= (4.*np.pi*d**2)
                ospec_nmo_tof[flavor][j]/= (4.*np.pi*d**2)
                #ospec_imo_tof[flavor][j] /= (4.*np.pi*d**2)
                
        for flavor in Flavor:
            ilum[flavor][i] = np.trapz(ispec[flavor].to('1/(erg*s)'), energies.to('erg')).value 
            olum_nmo[flavor][i] = np.trapz(ospec_nmo[flavor].to('1/(erg*s)'), energies.to('erg')).value
            olum_imo[flavor][i] = np.trapz(ospec_imo[flavor].to('1/(erg*s)'), energies.to('erg')).value
            olum_nmo_tof[flavor][i] = np.trapz(ospec_nmo_tof[flavor].to('1/(erg*s)'), energies.to('erg')).value
            #olum_imo_tof[flavor][i] = np.trapz(ospec_imo_tof[flavor].to('1/(erg*s)'), energies.to('erg')).value
            
    # make the figures 
    fig, axes = plt.subplots(3,3, figsize=(20,12), tight_layout=True)
    
    smax = [0.,0.,0.]
    titles = ['Untransformed', 'Transformed (NMO)', 'Transformed (nmo_TOF)','Transformed (IMO)','Transformed (imo_TOF)']
    titles = ['Untransformed','Transformed (NMO)','Transformed (nmo_TOF)']
    for i, spec in enumerate([ilum, olum_nmo, olum_nmo_tof]):
        for j, phase in enumerate([burst_epoch, accretion_epoch, cooling_epoch]):
            ax = axes[i,j]
            timeunits = 'ms' if j==0 else 's'
                
            for flavor in Flavor:
                if i == 0:
                    smax[j] = np.maximum(smax[j], 1.1*np.max(spec[flavor][phase]))
                    
                ax.plot(times[phase].to(timeunits),
                        spec[flavor][phase], label=flavor.to_tex(), lw=3,
                        color='C0' if flavor.is_electron else 'C1',
                        ls='-' if flavor.is_neutrino else ':')
            
            ax.set(xlim=(times[phase][0].to(timeunits).value, times[phase][-1].to(timeunits).value),
                   ylim=(0, smax[j]))
            
            if j==0:
               ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
               ax.legend(loc='upper right', ncol=1, fontsize=18)
            if j==1:
                ax.set(title=titles[i])
            if i < 2:
                ax.set(xticklabels=[])
            else:
                ax.set(xlabel='time [{}]'.format(timeunits))
            
            ax.grid(ls=':')

      
    return fig

In [ ]:
 energy_ranges = [(1, 10), (10, 30), (30, 50)]  # MeV
    # 定义三个能量范围，以MeV为单位
    D = 10 * u.kpc
    # 定义到超新星的距离，单位为千秒差距
    d = D.to('cm').value  # 到SN的距离
    # 将距离转换为厘米
    mass = 0.2 * u.eV
    # 定义中微子的质量，单位为电子伏特
    times = model.get_time()
    # 从模型中获取时间数组
    burst_epoch = times <= 0.1 * u.s
    # 定义爆发期为时间小于等于0.1秒
    accretion_epoch = (times > 0.1 * u.s) & (times <= 0.5 * u.s)
    # 定义吸积期为时间在0.1到0.5秒之间
    cooling_epoch = (times > 0.5 * u.s) & (times <= 10 * u.s)
    # 定义冷却期为时间在0.5到10秒之间
    
    ilum = {rng: {flavor: np.zeros(len(times)) for flavor in Flavor} for rng in energy_ranges}
    # 初始化字典以存储每个能量范围和味道的未转换光度
    olum_nmo = {rng: {flavor: np.zeros(len(times)) for flavor in Flavor} for rng in energy_ranges}
    # 初始化字典以存储每个能量范围和味道的NMO转换光度
    olum_imo = {rng: {flavor: np.zeros(len(times)) for flavor in Flavor} for rng in energy_ranges}
    # 初始化字典以存储每个能量范围和味道的IMO转换光度
    
    for i, t in enumerate(times):
        # 遍历每个时间步骤
        ispec = model.get_initial_spectra(t, energies)
        # 获取当前时间步骤的初始光谱
        ospec_nmo = model.get_transformed_spectra(t, energies, xform_nmo)
        # 获取当前时间步骤的NMO转换光谱
        ospec_imo = model.get_transformed_spectra(t, energies, xform_imo)
        # 获取当前时间步骤的IMO转换光谱
        
        for energy_range in energy_ranges:
            # 遍历每个能量范围
            e_min, e_max = energy_range
            # 解包当前范围的最小和最大能量
            mask = (energies.value >= e_min) & (energies.value <= e_max)
            # 创建一个掩码以选择当前范围内的能量
            
            for flavor in Flavor:
                # 遍历每种中微子味道
                ilum[energy_range][flavor][i] = np.trapz(ispec[flavor][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
                # 在当前能量范围内对初始光谱进行积分，并按距离平方进行归一化
                olum_nmo[energy_range][flavor][i] = np.trapz(ospec_nmo[flavor][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
                # 在当前能量范围内对NMO转换光谱进行积分，并按距离平方进行归一化
                olum_imo[energy_range][flavor][i] = np.trapz(ospec_imo[flavor][mask].to('1/(erg*s)'), energies[mask].to('erg')).value / (4 * np.pi * d ** 2)
                # 在当前能量范围内对IMO转换光谱进行积分，并按距离平方进行归一化
    
    fig, axes = plt.subplots(3, 3, figsize=(20, 12), tight_layout=True)
    # 创建一个包含3x3子图的图形，并设置图形大小和布局
    
    smax = [0., 0., 0.]
    # 初始化列表以存储每个时期的最大通量值
    titles = ['Untransformed', 'Transformed (NMO)', 'Transformed (IMO)']
    # 定义图表标题
    
    for idx, spec in enumerate([ilum, olum_nmo, olum_imo]):
        # 遍历每种光谱类型（未转换、NMO、IMO）
        for jdx, phase in enumerate([burst_epoch, accretion_epoch, cooling_epoch]):
            # 遍历每个时期（爆发期、吸积期、冷却期）
            ax = axes[idx, jdx]
            # 获取当前子图轴
            timeunits = 'ms' if jdx == 0 else 's'
            # 将时间单位设置为毫秒（对于爆发期）或秒（对于其他时期）
                
            for energy_range in energy_ranges:
                # 遍历每个能量范围
                e_min, e_max = energy_range
                # 解包当前范围的最小和最大能量
                for flavor in Flavor:
                    # 遍历每种中微子味道
                    if idx == 0:
                        # 对于未转换光谱
                        smax[jdx] = np.maximum(smax[jdx], 1.1 * np.max(spec[energy_range][flavor][phase]))
                        # 更新当前时期的最大通量值
                        
                    ax.plot(times[phase].to(timeunits),
                            spec[energy_range][flavor][phase], label=f"{flavor.to_tex()} ({e_min}-{e_max} MeV)", lw=3,
                            color='C0' if flavor.is_electron else 'C1',
                            ls='-' if flavor.is_neutrino else ':')
                    # 绘制当前味道和能量范围的通量图
            
            ax.set(xlim=(times[phase][0].to(timeunits).value, times[phase][-1].to(timeunits).value),
                   ylim=(0, smax[jdx]))
            # 设置当前子图的x轴和y轴限制
            
            if jdx == 0:
                ax.set(ylabel=r'flux [$10^{16}$ erg$^{-1}$ cm$^{-2}$ s$^{-1}$]')
                ax.legend(loc='upper right', ncol=1, fontsize=18)
                # 设置y轴标签，并在右上角添加图例
            if jdx == 1:
                ax.set(title=titles[idx])
                # 为中间列设置标题
            if idx < 2:
                ax.set(xticklabels=[])
                # 对前两行移除x轴刻度标签
            else:
                ax.set(xlabel='time [{}]'.format(timeunits))
                # 为最后一行设置x轴标签
            
            ax.grid(ls=':')
            # 在当前子图中添加网格线

    return fig
    # 返回图形